# TCR-CAR antagonism model

Notebook to explore the TCR/CAR antagonism model solutions. This notebook focuses on plotting the different variables at steady-state for the best parameter fits, without comparing to data for the moment. To manually vary parameters and see the effects on antagonism compared to data, use the notebook `notebooks_model_analysis/manual_tcr_car_fitting_tests.ipynb`. See also `notebooks_model_analysis/model_interpretation_6f.ipynb` for a closer look at 6F TCRs in TCR and TCR/CAR antigen mixtures. 

Section **Output contributions as a function of $L^T$** generates the model outputs plotted in figure S2G. 

## Model equations
Solving for steady-state, we find the following expressions for the numbers of bound TCRs in various proofreading stages $n$, the numbers of bounds CARs in various stages, and the numbers of activated inhibitory molecules by each receptor type. 

The numbers of receptors in each proofreading state are, for each receptor type $\rho$,

$$ C^{\rho}_n = \frac{R^{\rho}_b}{\varphi^{\rho} \tau^{\rho} + 1} (\Phi^{\rho})^n \quad (0 \leq n < N^{\rho} - f^{\rho}) $$
$$ C^{\rho}_n = \frac{R^{\rho}_b}{\psi^{\rho}(\vec{I}) \tau^{\rho} + 1} (\Phi^{\rho})^{N^{\rho} - f^{\rho}} (\Phi^{\rho}_{\vec{I} })^{n - N^{\rho} + f^{\rho}} \quad (N^{\rho} - f^{\rho} \leq n < N^{\rho}) $$
$$ C^{\rho}_{N^{\rho}} = R^{\rho}_b (\Phi^{\rho})^{N^{\rho} - f^{\rho}} (\Phi^{\rho}_{\vec{I}})^{f^{\rho}}   $$

where we again defined the regular and inhibited proofreading factors

$$ \Phi^{\rho} = \left( \frac{\varphi^{\rho} \tau^{\rho}}{\varphi^{\rho} \tau^{\rho} + 1}\right) $$ 
$$ \Phi^{\rho}_{\vec{I}} = \left( \frac{\psi^{\rho}(\vec{I}) \tau^{\rho}}{\psi^{\rho}(\vec{I}) \tau^{\rho} + 1}\right) $$

with 

$$     \psi^{\rho}(\vec{I}) = \varphi^{\rho} \frac{(I^{\rho}_{\mathrm{th}})^{k^{\rho}_I}}{(I^{\rho}_{\mathrm{th}})^{k^{\rho}_I} + (\sum_{\mu} {\gamma^{\rho}}_{\mu} I^{\mu})^{k^{\rho}_I}} + \psi^{\rho}_0 \,\, .$$

Each form $I^{\rho}$ is activated out of a total pool of inhibitory molecules $I_{\mathrm{tot}}$ by the complexes $C^{\rho}_{m^{\rho}}$, such that

$$ I^{\rho} = I_{tot} \frac{C^{\rho}_{m^{\rho}} / C^{\rho}_{m, \mathrm{th}}}{1 + \sum_{\mu} (C^{\mu}_{m^{\mu}} / C^{\mu}_{m, \mathrm{th}})}  \,\,. 
$$ 

The total number of bound receptors are given by quadratic formulas,

$$ R^T_b = \sum_{n=0}^{N^T} C^T_n = \frac12 \left(L^T + R^T + \frac{1}{\kappa^T \tau^T}\right) - \frac12 \sqrt{\left(L^T + R^T + \frac{1}{\kappa^T \tau^T}\right)^2 - 4 L^T R^T} $$

$$ R^C_b = \sum_{n=0}^{N^C} C^C_n = \frac12 \left(L^C + R^C + \frac{1}{\kappa^C \tau^C}\right) - \frac12 \sqrt{\left(L^C + R^C + \frac{1}{\kappa^C \tau^C}\right)^2 - 4 L^C R^C} \,\, .$$

The $\gamma$ matrix controls how each receptor type inhibits itself and the other. Note that equations for $I^T$ and $I^C$ are implicit if either $C^T_{m_T}$ or $C^C_{m_C}$ depend on $I$, which happens if $m^T \geq N^T - f^T$ or $m^C \geq N^C - f^C$. Unless that happens, the feedback is in reality an incoherent feedforward interaction and the solution is completely analytical. 

Lastly, the fully activated TCR and CAR numbers, $C^T_{N^T}$ and $C^C_{N^C}$, are passsed in a Michaelis-Menten activation function, with the threshold set by the receptor outputs of a large number of ligands of threshold binding times $\tau^T_c$ $\tau^C_c$ (these threshold $\tau$ are two fitted parameters in the MCMC simulations). 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
import json, h5py
import os, sys
if "../" not in sys.path:
    sys.path.insert(1, "../")

In [ ]:
from models.tcr_car_akpr_model import (
    activation_function, steady_akpr_i_receptor_types, steady_akpr_i_1ligand, psi_of_i_gamma, 
)
from mcmc.costs_tcr_car_antagonism import repackage_tcr_car_params
from models.akpr_i_model import psi_of_i
from mcmc.plotting import handles_properties_legend

In [ ]:
# Option to save some generated plots or not
do_save = False
do_save_outputs = False 

fig_folder = os.path.join("..", "figures", "model_details")
mcmc_folder = os.path.join("..", "results", "mcmc")
res_folder = os.path.join("..", "results", "for_plots")

# Display figures larger
plt.rcParams['figure.dpi'] = 150 # default for me was 75
plt.rcParams["font.family"] = "Arial"

# Code taken from 
# https://stackoverflow.com/questions/49223702/adding-a-legend-to-a-matplotlib-plot-with-a-multicolored-line
from matplotlib.legend_handler import HandlerLineCollection
from matplotlib.collections import LineCollection

class HandlerColorLineCollection(HandlerLineCollection):
    def create_artists(self, legend, artist ,xdescent, ydescent,
                        width, height, fontsize,trans):
        x = np.linspace(0,width,self.get_numpoints(legend)+1)
        y = np.zeros(self.get_numpoints(legend)+1)+height/2.-ydescent
        points = np.array([x, y]).T.reshape(-1, 1, 2)
        segments = np.concatenate([points[:-1], points[1:]], axis=1)
        lc = LineCollection(segments, cmap=artist.cmap,
                     transform=trans)
        lc.set_array(x)
        lc.set_linewidth(artist.get_linewidth())
        return [lc]

## Model parameters
Fitted values on TCR-TCR antagonism, then CAR-TCR antagonism. 

The best parameter fit is for (1, 2, 1). 

In [ ]:
with open(os.path.join("..", "data", "pep_tau_map_ot1.json"), "r") as handle:
    pep_tau_map = json.load(handle)

In [ ]:
# Load best parameter fit for (1, 2, 1)
fit_conc = ["1uM", "1nM"]
analysis_res_fname = "mcmc_analysis_tcr_car_both_conc.json"
with open(os.path.join(mcmc_folder, analysis_res_fname), "r") as jfile:
    all_results_dicts = json.load(jfile)
    del jfile

# Go back to linear-scale parameters
chosen_kmf = (1, 2, 1)
pvec_best = np.asarray(all_results_dicts.get(str(chosen_kmf)).get("param_estimates").get("MAP best"))

# Load constant parameter values
samples_fname = samples_fname = "mcmc_results_tcr_car_both_conc.h5"
with h5py.File(os.path.join(mcmc_folder, samples_fname), "r") as rfile:
    data_group = rfile.get("data")
    fit_param_names = list(rfile.get("samples").attrs.get("param_names"))
    l_conc_mm_params = data_group.get("l_conc_mm_params")[()]
    cost_args_loaded = [data_group.get(a)[()]
                        for a in data_group.attrs.get("cost_args_names")]
    del data_group, rfile

In [ ]:
### Rearrange loaded parameters in the correct format 
other_rates, ritot, nmf_fixed, cd19_tau_l = cost_args_loaded
res = repackage_tcr_car_params(pvec_best, chosen_kmf, other_rates, ritot, nmf_fixed)
(all_rates, tcr_rates, car_rates, ritot_vec, 
    tcr_ri, car_ri, nmf_both, tcr_nmf, car_nmf, threshold_taus) = res

In [ ]:
# Compute thresholds
# Evaluate CAR threshold; TCR is fixed and received as parameter
tcr_thresh = steady_akpr_i_1ligand(tcr_rates, threshold_taus[0],
        10*tcr_ri[0], tcr_ri, tcr_nmf, large_l=True)[tcr_nmf[0]]
car_thresh = steady_akpr_i_1ligand(car_rates, threshold_taus[1],
        10*car_ri[0], car_ri, car_nmf, large_l=True)[car_nmf[0]]

# Agonist alone output
ag_alone = steady_akpr_i_1ligand(car_rates, *cd19_tau_l, car_ri, car_nmf)[car_nmf[0]]
ag_alone = activation_function(ag_alone, car_thresh)

In [ ]:
# Print the best fit parameters
for i, p in enumerate(fit_param_names):
    print(p, ":", 10.0**pvec_best[i])

# Model analysis

## Basic plotting of steady-state solutions

We are interested in showing the value of $\psi(\vec{I})$ at steady-state as well, since the outputs depend on this rate where the negative feedback acts. 

Note that $Z^C$ (the CAR's contribution to the total output) is inhibited by the TCR and remains so even in the presence of TCR agonists. This is probably unrealistic, the real biological circuit probably includes a positive feedback as well, which shuts down the inhibitory feedback and creates synergy between receptors. But we did not focus on modelling this positive interaction, and instead went for the simplest model that could fit the antagonism effect. 

### TCR output alone

In [ ]:
# Plot all complexes and all SHP-1s as a function of L_TCR for a few tau_TCR
#tau_tcr_range2 = np.sort(np.asarray(list(pep_tau_map.values())))
#l_tcr_range2 = np.logspace(1.0, 5.0, 101)
tau_tcr_range = np.linspace(0.001, 10.0, 101)
l_tcr_range = np.asarray([10, 100, 1000, 10000, 100000])

model_columns_tcr = pd.Index([r"$C^T_{}$".format(n) for n in range(tcr_nmf[0]+1)]
                   + [r"$I$", r"$Z^T$"], name="Variable")
model_index_tcr = pd.MultiIndex.from_product([l_tcr_range, tau_tcr_range], names=[r"$L^T$", r"$\tau^T$"])
df_model_tcr = pd.DataFrame(np.zeros([len(model_index_tcr), len(model_columns_tcr)]), 
                       columns=model_columns_tcr, index=model_index_tcr)

tcr_rates_arr = [np.asarray([a]) for a in tcr_rates]
tcr_rates_arr.insert(5, np.ones(1))
tcr_ri_arr = [np.asarray([tcr_ri[0]]), tcr_ri[1]]
tcr_nmf_arr = [np.asarray([a]) for a in tcr_nmf]
for l_tcr, tau_tcr in model_index_tcr:
    complexes = steady_akpr_i_1ligand(tcr_rates, tau_tcr, l_tcr, tcr_ri, tcr_nmf)
    tau_arr = np.asarray([tau_tcr])
    l_arr = np.asarray([l_tcr])
    
    # Compare to receptors types output for a single type. Should be the same, assert to be sure. 
    complexes_mix = steady_akpr_i_receptor_types(tcr_rates_arr, tau_arr, l_arr, tcr_ri_arr, tcr_nmf_arr)
    assert np.allclose(complexes[:-1] - complexes_mix[0], 0.0)
    assert np.allclose(complexes[-1] - complexes_mix[1], 0.0)

    df_model_tcr.loc[(l_tcr, tau_tcr), r"$C^T_0$":r"$C^T_{}$".format(tcr_nmf[0])] = complexes[:-1]
    df_model_tcr.loc[(l_tcr, tau_tcr), r"$I$"] = complexes[-1]
    # Normalize outputs to compare CAR and TCR properly, accounting for
    # their very different signaling potencies.
    df_model_tcr.loc[(l_tcr, tau_tcr), r"$Z^T$"] = activation_function(complexes[-2], tcr_thresh)

for k in df_model_tcr.index:
    psi = psi_of_i(df_model_tcr.loc[k, r"$I$"], tcr_rates[3], tcr_rates[4], tcr_rates[0], tcr_rates[5])
    df_model_tcr.loc[k, r"$\psi$"] = psi

In [ ]:
df_plot = df_model_tcr + df_model_tcr.max(axis=0)*1e-4
ytitles = [r"$C^T_2$", r"$I$", r"$\psi$", r"$C^T_6$"]
df_plot = df_plot.loc[:, ytitles]
df_plot = df_plot.stack().reset_index()
df_plot[r"$L^T$"] = df_plot[r"$L^T$"].astype(str)
g = sns.relplot(data=df_plot, x=r"$\tau^T$", y=0, col="Variable", hue=r"$L^T$", 
            height=2.0, col_wrap=len(ytitles), kind="line", facet_kws=dict(sharey=False), 
            hue_order=l_tcr_range[::-1].astype(str), palette="viridis")
for i, ax in enumerate(g.axes.flat):
    ax.set_yscale("log")
    ax.set_ylabel(ytitles[i])
g.tight_layout()
plt.show()
plt.close()

### TCR and CAR outputs combined

In [ ]:
# Plot all complexes and all inhibitory species I^\rho as a function of tau^T and L^T in presence of CD19. 
# Also plot the psi functions. See how all these quantities vary and saturate. 
tau_tcr_range = np.linspace(0.001, 10.0, 101)
l_tcr_range = np.asarray([10, 100, 1000, 10000, 100000])

model_columns = pd.Index([r"$C^T_{}$".format(n) for n in range(tcr_nmf[0]+1)] 
                   + [r"$C^C_{}$".format(n) for n in range(car_nmf[0]+1)]
                   + [r"$I^T$", r"$I^C$", r"$Z^T$", r"$Z^C$", "Ratio"], name="Variable")
model_index = pd.MultiIndex.from_product([l_tcr_range, tau_tcr_range], names=[r"$L^T$", r"$\tau^T$"])
df_model = pd.DataFrame(np.zeros([len(model_index), len(model_columns)]), 
                       columns=model_columns, index=model_index)

for l_tcr, tau_tcr in model_index:
    taus = np.asarray([tau_tcr, cd19_tau_l[0]])
    lvec = np.asarray([l_tcr, cd19_tau_l[1]])
    complexes_mix = steady_akpr_i_receptor_types(all_rates, taus, lvec, ritot_vec, nmf_both)
    df_model.loc[(l_tcr, tau_tcr), r"$C^T_0$":r"$C^T_{}$".format(tcr_nmf[0])] = complexes_mix[0]
    df_model.loc[(l_tcr, tau_tcr), r"$C^C_0$":r"$C^C_{}$".format(car_nmf[0])] = complexes_mix[1]
    df_model.loc[(l_tcr, tau_tcr), r"$I^T$":r"$I^C$"] = complexes_mix[2]
    # Normalize outputs to compare CAR and TCR properly, accounting for
    # their very different signaling potencies.
    df_model.loc[(l_tcr, tau_tcr), r"$Z^T$"] = activation_function(complexes_mix[0][-1], tcr_thresh)
    df_model.loc[(l_tcr, tau_tcr), r"$Z^C$"] = activation_function(complexes_mix[1][-1], car_thresh)
    df_model.loc[(l_tcr, tau_tcr), "Ratio"] = (df_model.loc[(l_tcr, tau_tcr), r"$Z^T$":r"$Z^C$"].sum() 
                                                    / ag_alone)

for k in df_model.index:
    psis = psi_of_i_gamma(df_model.loc[k, r"$I^T$":r"$I^C$"].values, 
                      all_rates[3], all_rates[4], all_rates[0], all_rates[5], all_rates[6])
    df_model.loc[k, r"$\psi^T$"] = psis[0]
    df_model.loc[k, r"$\psi^C$"] = psis[1]

In [ ]:
df_plot = df_model + df_model.max(axis=0)*1e-4
df_plot = df_plot.loc[:, r"$I^T$":]
df_plot = df_plot.stack().reset_index()
df_plot[r"$L^T$"] = df_plot[r"$L^T$"].astype(str)

g = sns.relplot(data=df_plot, x=r"$\tau^T$", y=0, col="Variable", hue=r"$L^T$", 
               height=2.0, col_wrap=4, kind="line", facet_kws=dict(sharey=False), 
               hue_order=l_tcr_range[::-1].astype(str), palette="magma")
for ax in g.axes.flat:
    ax.set_yscale("log")
plt.show()
plt.close()

# Output contributions as a function of $L^T$, for a few typical TCR antigen strengths
Show agonist, partial agonist, antagonist

This part generates the underlying model results for supplementary figure S2G. 

In [ ]:
# Plot all complexes and all SHP-1s as a function of tau_tcr and L_TCR in presence of CD19. 
# Also plot the psi functions. See how all that stuff varies and saturates. 
tau_tcr_range2 = np.asarray([2.0, 4.0, 10.0])
l_tcr_range2 = np.logspace(0, 5, 201)

model_columns2 = pd.Index([r"$T_{}$".format(n) for n in range(tcr_nmf[0]+1)] 
                   + [r"$C_{}$".format(n) for n in range(car_nmf[0]+1)]
                   + [r"$I^T$", r"$I^C$", r"$Z^T$", r"$Z^C$", "Ratio"], name="Variable")
model_index2 = pd.MultiIndex.from_product([l_tcr_range2, tau_tcr_range2], names=[r"$L^T$", r"$\tau^T$"])
df_model2 = pd.DataFrame(np.zeros([len(model_index2), len(model_columns2)]), 
                       columns=model_columns2, index=model_index2)

for l_tcr, tau_tcr in model_index2:
    taus = np.asarray([tau_tcr, cd19_tau_l[0]])
    lvec = np.asarray([l_tcr, cd19_tau_l[1]])
    complexes_mix = steady_akpr_i_receptor_types(all_rates, taus, lvec, ritot_vec, nmf_both)
    df_model2.loc[(l_tcr, tau_tcr), r"$T_0$":r"$T_{}$".format(tcr_nmf[0])] = complexes_mix[0]
    df_model2.loc[(l_tcr, tau_tcr), r"$C_0$":r"$C_{}$".format(car_nmf[0])] = complexes_mix[1]
    df_model2.loc[(l_tcr, tau_tcr), r"$I^T$":r"$I^C$"] = complexes_mix[2]
    # Normalize outputs to compare CAR and TCR properly, accounting for
    # their very different signaling potencies.
    df_model2.loc[(l_tcr, tau_tcr), r"$Z^T$"] = activation_function(complexes_mix[0][-1], tcr_thresh)
    df_model2.loc[(l_tcr, tau_tcr), r"$Z^C$"] = activation_function(complexes_mix[1][-1], car_thresh)
    df_model2.loc[(l_tcr, tau_tcr), "Ratio"] = (df_model2.loc[(l_tcr, tau_tcr), r"$Z^T$":r"$Z^C$"].sum() 
                                                    / ag_alone)

In [ ]:
def strip_names(df, char, axis=0):
    stripper = lambda x: x.strip(char)
    return df.rename(mapper=stripper, axis=axis)

def pad_names(df, char, axis=0):
    padder = lambda x: char + x + char
    return df.rename(mapper=padder, axis=axis)

In [ ]:
df_model_save = strip_names(df_model2, "$", axis=1)
df_model_save.index = df_model_save.index.set_names([a.strip("$").strip("\\").replace("^", "_") 
                                                     for a in df_model_save.index.names])
if do_save_outputs:
    strip_names(df_model_save, "$", axis=1).to_hdf(
        os.path.join(res_folder, "output_contributions.h5"), key="df")
    with h5py.File(os.path.join(res_folder, "output_contributions.h5"), "a") as h:
        try:
            grp = h.create_group("ag_alone")
        except ValueError as e:
            print(e)
        else:
            grp["Z_C"] = ag_alone
            del grp

In [ ]:
# Two subplots, one for each TCR tau. Shared legend. 
# New version with 4 mini-cartoons. Two plots on top of each other. 
tcr_ags_palette = ["#4c78bbff", "#d83ad8ff", "#DA3833"]  # antagonist, partial agonist, agonist
car_cd19_color = "grey"
labelsize = 8
antagonism_palette = np.asarray(sns.color_palette('PuOr_r', n_colors=100))[[13, 87]]  # Antagonism, enhancement
print(sns.color_palette(antagonism_palette).as_hex()[:])
palette = list(sns.color_palette("Greys", n_colors=2))
fig, axes = plt.subplots(3, 1)
axes = axes.flatten()
fig.set_size_inches(3.0, 1.8*2)
for i, tcr_tau in enumerate(tau_tcr_range2):
    ax = axes[i]
    y_c = df_model2.loc[(l_tcr_range2, tcr_tau), r"$Z^C$"]
    y_t = df_model2.loc[(l_tcr_range2, tcr_tau), r"$Z^T$"]
    y_tot = (y_c + y_t) / ag_alone
    y_c = y_c / ag_alone
    y_t = y_t / ag_alone
    
    # plot the same data on both axes
    # Code from https://stackoverflow.com/questions/38051922/how-to-get-differents
    # -colors-in-a-single-line-in-a-matplotlib-figure
    # select how to color
    cmap = mpl.colors.ListedColormap(antagonism_palette)
    norm = mpl.colors.BoundaryNorm([-np.inf, 1.0], cmap.N)
    # get segments
    xy = np.array([l_tcr_range2, y_tot]).T.reshape(-1, 1, 2)
    segments = np.hstack([xy[:-1], xy[1:]])
    # make line collection
    lc = mpl.collections.LineCollection(segments, cmap=cmap, norm=norm, 
                                        label="Total", lw=2.5)
    lc.set_array(y_tot)
    ax.add_collection(lc)
    ligand_type = "antagonist" if i == 0 else "partial agonist"
    ax.plot(l_tcr_range2, y_c, label=r"$Z^C$", color=palette[1], ls=":", lw=2.0)
    ax.plot(l_tcr_range2, y_t, label=r"$Z^T$", color=tcr_ags_palette[i], ls="--", lw=2.0)
    
    #ax.set_ylim(-0.1, df_model2["Ratio"].max()*1.1)
    ax.set_xlabel("TCR Ag density (#)", size=labelsize, labelpad=0.1)
    ax.set_xscale("log")
    ax.axhline(1.0, ls='-', color="k", lw=0.75)
    ax.set_ylabel(r"$Z(\mathrm{mix})\,/\,Z^C(\mathrm{CD19})$", size=labelsize, labelpad=0.2)
    # Create final legend inside the first plot
    if i == 1:
        # Change order of legend
        handles, labels = ax.get_legend_handles_labels()
        labels.insert(0, labels.pop())
        handles.insert(0, handles.pop())
        # Change color of the Z_T line before drawing legend
        back_color = handles[2].get_color()
        handles[2].set_color(handles[1].get_color())

        ax.legend(handles=handles, labels=labels, 
                  handler_map={lc: HandlerColorLineCollection(numpoints=2)},
                  loc="center right", bbox_to_anchor=(-0.15, 0.4), frameon=False,
                  #frameon=True, edgecolor=(1, 1, 1, 0), facecolor=(1, 1, 1, 0.9), framealpha=0.9, 
                  #labelspacing=0.2, handletextpad=0.4, columnspacing=0.5,  borderaxespad=0.3, borderpad=0.2, 
                  fontsize=labelsize, title="Output", title_fontsize=labelsize)
        # Restor line color on the plot
        handles[2].set_color(back_color)
    
    # Force more log ticks
    locmaj = mpl.ticker.LogLocator(base=10,numticks=5) 
    ax.xaxis.set_major_locator(locmaj)
    # Force minor log ticks
    locmin = mpl.ticker.LogLocator(base=10.0,subs=np.arange(0.1, 1.0, 0.1), numticks=5)
    ax.xaxis.set_minor_locator(locmin)
    ax.xaxis.set_minor_formatter(mpl.ticker.NullFormatter())
    # Change label size
    ax.tick_params(axis='both', which='major', labelsize=labelsize*0.85, pad=0.1, length=3.0)
    ax.tick_params(axis='both', which='minor', pad=0.1, length=2.0)
    
    # Hide unnecessary spines to make room for diagrams. 
    for s in ["top", "right"]:
        ax.spines[s].set_visible(False)

fig.tight_layout(w_pad=0.3, h_pad=0.3)
if do_save:
    fig.savefig(os.path.join(fig_folder, "akpr_i_output_contributions.pdf"), 
                bbox_inches="tight", transparent=True)
plt.show()
plt.close()

## Output contributions as a function of $\tau^T$

In [ ]:
# Plot I^T, I^C on the same graph, to show tradeoff. Maybe plot psi^c on a second axis? 
# Plot Z^T, Z^C, and their total on the same graph. For two Ls. Maybe normalize by Z^C for CAR alone. 
choice_l = [100000, 10000]
palette = sns.color_palette(n_colors=len(choice_l))
palette = {choice_l[i]:palette[i] for i in range(len(choice_l))}

fig, axes = plt.subplots(3)
fig.set_size_inches(4, 2*3)
axes = axes.flatten()

# Plot the S variables
ax = axes[0]
styles = {r"$I^T$":("-", None), r"$I^C$":("--", None)}#, r"$S_T + S_C$":("-", None)}  # for T, C
ylow = np.inf
for i in range(2):
    y_c = df_model.loc[(choice_l[i], tau_tcr_range), r"$I^C$"]
    if y_c.min() < ylow:
        ylow = y_c.min()
    y_t = df_model.loc[(choice_l[i], tau_tcr_range), r"$I^T$"]
    ax.plot(tau_tcr_range, y_c, color=palette[choice_l[i]], linestyle=styles[r"$I^T$"][0])
    ax.plot(tau_tcr_range, y_t, color=palette[choice_l[i]], linestyle=styles[r"$I^C$"][0])
    #ax.plot(tau_tcr_range, y_t + y_c, color=palette[choice_l[i]], linestyle=styles[r"$S_T + S_C$"][0])

# Custom legend
hues = (r"$L^T$", palette)
styles = ("Receptor type", styles)
legend_handles, legend_handler_map = handles_properties_legend(hues, styles, None)
ax.legend(handles=legend_handles, handler_map=legend_handler_map, 
          loc="upper left", bbox_to_anchor=(1.0, 1.0))
ylims = ax.get_ylim()
ax.set(yscale="log", xlabel=r"$\tau^T$", ylabel=r"Active $I$", ylim=(ylow*0.75, ylims[1]))

# Plot the psi variables
ax = axes[1]
styles = {r"$\psi^T$":("-", None), r"$\psi^C$":("--", None)}
for i in range(2):
    y_c = df_model.loc[(choice_l[i], tau_tcr_range), r"$\psi^C$"]
    y_c = y_c / y_c.iloc[0]
    y_t = df_model.loc[(choice_l[i], tau_tcr_range), r"$\psi^T$"]
    y_t = y_t / y_t.iloc[0]
    ax.plot(tau_tcr_range, y_c, color=palette[choice_l[i]], linestyle=styles[r"$\psi^C$"][0])
    ax.plot(tau_tcr_range, y_t, color=palette[choice_l[i]], linestyle=styles[r"$\psi^T$"][0])
    
# Custom legend
hues = (r"$L^T$", palette)
styles = ("Receptor type", styles)
legend_handles, legend_handler_map = handles_properties_legend(hues, styles, None)
ax.legend(handles=legend_handles, handler_map=legend_handler_map, 
          loc="upper left", bbox_to_anchor=(1.0, 1.0))
ax.set(yscale="log", xlabel=r"$\tau^T$", ylabel=r"$\psi\, /\, \psi(\tau^T = 0)$")


# Plot the Z variables
ax = axes[2]
styles = {r"$Z^T$":("--", None), r"$Z^C$":(":", None), r"$Z_{tot}$":("-", None)}  # for T, C, and total

ylow = np.inf
for i in range(2):
    y_c = df_model.loc[(choice_l[i], tau_tcr_range), r"$Z^C$"]
    y_t = df_model.loc[(choice_l[i], tau_tcr_range), r"$Z^T$"]
    if y_c.min() < ylow:
        ylow = y_c.min()
    ax.plot(tau_tcr_range, y_c, color=palette[choice_l[i]], linestyle=styles[r"$Z^T$"][0])
    ax.plot(tau_tcr_range, y_t, color=palette[choice_l[i]], linestyle=styles[r"$Z^C$"][0])
    ax.plot(tau_tcr_range, y_t + y_c, color=palette[choice_l[i]], linestyle=styles[r"$Z_{tot}$"][0])

# Custom legend
hues = (r"$L^T$", palette)
styles = ("Output", styles)
legend_handles, legend_handler_map = handles_properties_legend(hues, styles, None)
ax.legend(handles=legend_handles, handler_map=legend_handler_map, 
          loc="upper left", bbox_to_anchor=(1.0, 1.0))
ylims = ax.get_ylim()
ax.set(yscale="log", xlabel=r"$\tau^T$", ylabel=r"Normalized output $Z$", ylim=(ylow*0.75, ylims[1]))

fig.tight_layout()
plt.show()
plt.close()

# Inhibitory species $I$ as a function of $\tau^T_l$
The more relevant plot would be as a function of both $\tau$s in TCR/TCR antagonism (see `tcr_tcr_akpr_model_interpretation.ipynb`), but we will here examine the activated $I^\rho$ species as a function of antigen parameters in TCR/CAR antagonism. 

In [ ]:
# Plot I as a function of tau_tcr and tau_car
# Also plot the psi functions. See how all that stuff varies and saturates. 
taut_range = np.linspace(0.001, 20.0, 100)
tauc_range = np.linspace(1.0, 1000.0, 100)
taut_grid, tauc_grid = np.meshgrid(taut_range, tauc_range, indexing="ij")  # tau2 on the x axis
lt_tcr = l_conc_mm_params[0]
lc_car = cd19_tau_l[1]

mat_itc = pd.DataFrame(np.zeros([taut_range.size*tauc_range.size, 2]), 
            index=pd.MultiIndex.from_product([taut_range, tauc_range], names=["tau_T", "tau_C"]), 
            columns=pd.Index(["I_T", "I_C"], name="I_kind"))
lvec = np.asarray([lt_tcr, lc_car])
for i in range(taut_range.size):
    for j in range(tauc_range.size):
        taus = np.asarray([taut_range[i], tauc_range[j]])
        complexes_mix = steady_akpr_i_receptor_types(all_rates, taus, lvec, ritot_vec, nmf_both)
        mat_itc.loc[(taut_range[i], tauc_range[j]), "I_T":"I_C"] = complexes_mix[-1]
display(mat_itc)

In [ ]:
# Two subplots to show I_T and I_C?
fig, axes = plt.subplots(1, 2)
fig.set_size_inches(fig.get_size_inches()[0]*1.25, fig.get_size_inches()[1]*1.25/2.0)
# I_T plot
im = axes[0].imshow(mat_itc["I_T"].unstack("tau_C").values[::-1, :], cmap=plt.cm.Greens, 
    extent=(tauc_range[0]/cd19_tau_l[0], tauc_range[-1]/cd19_tau_l[0], 
            taut_range[0]/pep_tau_map["N4"], taut_range[-1]/pep_tau_map["N4"]), aspect="equal")
divider = make_axes_locatable(axes[0])
cax = divider.append_axes("right", size="8%", pad=0.1)
cbar = fig.colorbar(im, cax=cax)
cbar.set_label(r"$I^T$", labelpad=3)
axes[0].set(xlabel=r"$\tau^C/\tau^C_{\mathrm{CD19}}$", ylabel=r"$\tau^T/\tau^T_{\mathrm{N4}}$")

# I_C plot
imc = axes[1].imshow(mat_itc["I_C"].unstack("tau_C").values[::-1, :], cmap=plt.cm.Blues,
    extent=(tauc_range[0]/cd19_tau_l[0], tauc_range[-1]/cd19_tau_l[0], 
            taut_range[0]/pep_tau_map["N4"], taut_range[-1]/pep_tau_map["N4"]), aspect="equal")
divider = make_axes_locatable(axes[1])
cax = divider.append_axes("right", size="8%", pad=0.1)
cbar = fig.colorbar(imc, cax=cax)
cbar.set_label(r"$I^C$", labelpad=3)
axes[1].set(xlabel=r"$\tau^C/\tau^C_{\mathrm{CD19}}$", ylabel=r"$\tau^T/\tau^T_{\mathrm{N4}}$")

fig.tight_layout()
if do_save:
    fig.savefig(os.path.join(fig_folder, "inhibitory_species_vs_taus_tcr-car.pdf"), 
            transparent=True, bbox_inches="tight")
plt.show()
plt.close()